[lab2](https://drive.google.com/file/d/1uiras3KLbnL8s-nsLoxBYJMEEoejhWuJ/view)

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import deque
import math

Zapewne pamiętasz o odkurzaczu omówionym w [pierwszych dwóch wykładach](https://drive.google.com/file/d/1NrPKqnpIc1JBmXAK1CsjRdKHXk0CREM2/view).
(i) Sformułuj problem jako problem przeszukiwania; tzn. musisz napisać stan
początkowy, możliwe działania za pomocą funkcji Action(state), relację
przejścia za pomocą funkcji Result(state, action) i stan celu. Proszę zobacz
yć strona 9 wykładu 2. Zakładamy, że obie kwadraty A i B są brudne i
agent znajduje się w kwadracie A. Zakładamy również, że agent ma trzy

możliwe działania: ruszaj w lewo, ruszaj w prawo i ssać. (ii) Napisz pro-
gram za pomocą algorytm ‘Breadth First Search, aby agent mógł odszukiwać

brud we wszystkich możliwych lokalizacjach i wyczyścić go (tzn., znalezienie
ścieżkę z węzła początkowego do węzła docelowego. Wystarczy, aby program
wydrukował każdą sekwencję akcji na konsoli, i pokazał kiedy osiąga cel
(czyli ścieżka z węzła początkowego do węzła docelowego).

In [5]:
class VacuumCleaner:
    def __init__(self):
        self.initial_state = (0, ['dirty', 'dirty'])
        self.goal_state = ['clean', 'clean']

    def actions(self, state):
        position, dirt_status = state
        possible_actions = []
        
        if dirt_status[position] == 'dirty':
            possible_actions.append('suck')
        if position == 0:
            possible_actions.append('move right')
        elif position == 1:
            possible_actions.append('move left')
        
        return possible_actions
    
    def result(self, state, action):
        position, dirt_status = state
        new_dirt_status = dirt_status[:]
        
        if action == 'suck':
            new_dirt_status[position] = 'clean'
        elif action == 'move right':
            return (1, new_dirt_status)
        elif action == 'move left':
            return (0, new_dirt_status)
    
        return (position, new_dirt_status)
    
    def bfs(self):
        queue = deque([(self.initial_state, [])])
        visited = set()
        
        while queue:
            state, path = queue.popleft()
            state_hashable = (state[0], tuple(state[1]))
            
            if state[1] == self.goal_state:
                # print("Solution found:", path)
                return path
            
            visited.add(state_hashable)
            
            for action in self.actions(state):
                new_state = self.result(state, action)
                new_state_hashable = (new_state[0], tuple(new_state[1]))
                
                if new_state_hashable not in visited:
                    queue.append((new_state, path + [action]))
        
        print("No solution found")
        return None

In [6]:
vacuum = VacuumCleaner()
vacuum.bfs()

['suck', 'move right', 'suck']

Rozważ 8 zagadek ze stanem początkowym jako <br>
[ ][1][3]<br>
[4][2][5]<br>
[7][8][6].<br>

Stan docelowy to:<br> 
[1][2][3]<br>
[4][5][6]<br>
[7][8][ ]<br>

(i) Opisz sformułowanie problemu jako problem przeszukiwania.

1. Stan początkowy: Układ kafelków z pustą komórką (np. [ ][1][3] [4][2][5] [7][8][6]).
2. Stan docelowy: Uporządkowana wersja układanki (np. [1][2][3] [4][5][6] [7][8][ ]).
3. Stan: Każda możliwa konfiguracja kafelków, w tym pozycja pustego miejsca.
4. Działania: Przesuwanie kafelków na puste pole (w górę, w dół, w lewo lub w prawo).
5. Cel: Osiągnięcie stanu docelowego.

Problem rozwiązuje się poprzez przeszukiwanie przestrzeni stanów w celu znalezienia sekwencji ruchów prowadzących do rozwiązania.

(ii) Napisz program, używając przeszukiwania A∗

[(Zobacz numer wykładu 3)](https://drive.google.com/file/d/1UDmFNgLoo5_PbGj05fuLRJHXcQYCQ6He/view). Uważamy, że każde krok kosztuje 1, a zatem koszty osiągnięcia stanu n
z stanu początkowego (czyli g(n)) jest całkowita liczba kroków wymaganych
do osiągnięcia stanu n. Możesz użyć funkcji heurystycznej, gdzie h(stan) =
suma odległości bloku miasta (sum of city block distance). Wystarczy, aby program wydrukował każdą sekwencję akcji na konsoli, i pokazał kiedy osiąga cel.

(Uwaga: W celu wykonania (ii) byłoby dobrze stworzyć drzewo gry z pow-
iązaną wartością f(n) = g(n) + h(n) dla każdego węzła.)

In [7]:
import heapq

goal_state = ((1, 2, 3), (4, 5, 6), (7, 8, 0))

def manhattan_distance(state):
    distance = 0
    for r in range(3):
        for c in range(3):
            value = state[r][c]
            if value != 0: 
                goal_r, goal_c = divmod(value - 1, 3)
                distance += abs(goal_r - r) + abs(goal_c - c)
    return distance

def is_goal_state(state):
    return state == goal_state

def get_neighbors(state):
    neighbors = []
    for r in range(3):
        for c in range(3):
            if state[r][c] == 0:
                row, col = r, c
                break

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    for dr, dc in directions:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_state = [list(row) for row in state] 
            new_state[row][col], new_state[new_row][new_col] = new_state[new_row][new_col], new_state[row][col]
            neighbors.append(tuple(tuple(row) for row in new_state))
    return neighbors

def astar(start_state):
    open_list = []
    closed_list = set()
    
    start_h = manhattan_distance(start_state)
    heapq.heappush(open_list, (start_h, 0, start_state, [])) 

    while open_list:
        f, g, current_state, path = heapq.heappop(open_list)

        if is_goal_state(current_state):
            return path + [current_state]
        
        closed_list.add(current_state)

        for neighbor in get_neighbors(current_state):
            if neighbor not in closed_list:
                h = manhattan_distance(neighbor)
                heapq.heappush(open_list, (g + 1 + h, g + 1, neighbor, path + [current_state]))

    return None 

In [8]:
start_state = ((0, 1, 3), (4, 2, 5), (7, 8, 6))

solution_path = astar(start_state)

if solution_path:
    for step in solution_path:
        for row in step:
            print(row)
        print()
else:
    print("Brak rozwiązania")

(0, 1, 3)
(4, 2, 5)
(7, 8, 6)

(1, 0, 3)
(4, 2, 5)
(7, 8, 6)

(1, 2, 3)
(4, 0, 5)
(7, 8, 6)

(1, 2, 3)
(4, 5, 0)
(7, 8, 6)

(1, 2, 3)
(4, 5, 6)
(7, 8, 0)



Zapewne pamiętasz o Trywialnej grze rozważanej w wykładach [3](https://drive.google.com/file/d/1UDmFNgLoo5_PbGj05fuLRJHXcQYCQ6He/view) i [4](https://www.youtube.com/watch?v=xvFZjo5PgG0). Napisz
program, aby znaleźć optymalną strategię dla MAX za pomocą algorytmu
Maximin z przycinaniem [α, β] ([α, β]-prunning).

In [1]:
import math

def maximin_alpha_beta(node, depth, alpha, beta, maximizing_player, tree):
    if node not in tree:
        return node 
    
    if maximizing_player:
        max_eval = -math.inf
        for child in tree[node]:
            eval = maximin_alpha_beta(child, depth + 1, alpha, beta, False, tree)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break  
        return max_eval
    else:
        min_eval = math.inf
        for child in tree[node]:
            eval = maximin_alpha_beta(child, depth + 1, alpha, beta, True, tree)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break  
        return min_eval

tree = {
    'A': ['B', 'C', 'D'],
    'B': [3, 12, 8],
    'C': [2, 4, 6],
    'D': [14, 5, 2]
}


optimal_value = maximin_alpha_beta('A', 0, -math.inf, math.inf, False, tree)
print("Optymalna wartość dla MAX:", optimal_value)


Optymalna wartość dla MAX: 6
